In [1]:
from transformers import AutoTokenizer
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import torch.nn as nn
import os
import numpy as np
from transformers import BertForSequenceClassification, AdamW

/home/iticbcn/Documentos/IA/Natural/natural/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModel
from transformers import BertTokenizer, TFBertModel

model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
with open("el_quijote.txt", "r", encoding="utf-8") as file:
    contenido = file.read()



In [4]:
def tokenizar(contenido):
    tokens = tokenizer.encode(contenido,max_length=512,truncation=True)
    data = torch.tensor(tokens, dtype=torch.long)
    return data


In [5]:
class text_dataset(Dataset):
    def __init__(self, text, tokenizer, input_length=10, context_size=50):
        tokens = tokenizer.encode(text)
        self.sentences = [tokens[i*input_length:i*input_length+input_length] for i in range(len(tokens)//10)]
        
        self.sentences_per_unit = input_length-1
        self.context_size = context_size
                     
        
    def __len__(self):
        return len(self.sentences*self.sentences_per_unit)

    def __getitem__(self, idx):
        sentence = self.sentences[idx//self.sentences_per_unit]
        #print(idx%self.sentences_per_unit+1)
        x = sentence[0:idx%self.sentences_per_unit+1]
        y = sentence[idx%self.sentences_per_unit+1]

        if len(x) < self.context_size:
            for i in range(len(x),self.context_size):
                x.append(0)
        
        return torch.tensor(x), y

dataset = text_dataset(contenido, tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (1475 > 512). Running this sequence through the model will result in indexing errors


In [6]:
x, y = dataset[109]
print(len(x))
print(y)

train_dataloader = DataLoader(dataset, shuffle=True, batch_size=20)

50
3678


In [7]:
def create_train_test_datasets(dataset, split_ratio=0.8):
    train_size = int(split_ratio * len(dataset))
    test_size = len(dataset) - train_size
    train_data, test_data = random_split(dataset, [train_size, test_size])

    return train_data, test_data
def create_dataloader(training_data, test_data, batch_size):
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    return train_dataloader, test_dataloader



In [8]:
def select_device():
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    print(device)
    return device


In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self, vocab_size, embed_size=100, encoders=4, head=10):
        super(NeuralNetwork, self).__init__()

        # Embedding layer para convertir palabras en vectores densos
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Capa Transformer Encoder
        #encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=head, batch_first=True)
        #self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=encoders)

        # Decoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=4)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

        # input = [batch_size, context_length, d_model]
        
        # Capa de regularización
        self.dropout = nn.Dropout(0.1)

        self.flatt = nn.Flatten()

        # Activación ReLU
        self.relu = nn.ReLU()

        # Capa de salida (embed_size → num_classes)
        self.fc = nn.Linear(embed_size*50, vocab_size)  # Cambié hidden_size a embed_size

    def forward(self, x):
        x = self.embedding(x)  # Convierte tokens en vectores de tamaño embed_size
        x = self.transformer_encoder(x)  # Pasa por el Transformer Encoder
        #x = torch.mean(x, dim=1)  # Promediamos los embeddings de la secuencia para clasificación
        x = self.dropout(self.flatt(x))  # Aplicamos dropout
        x = self.relu(x)  # Aplicamos ReLU
        logits = self.fc(x)  # Capa de clasificación final
        return logits

model = NeuralNetwork(tokenizer.vocab_size)
print(model.to("cuda"))


/home/iticbcn/Documentos/IA/Natural/natural/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


NeuralNetwork(
  (embedding): Embedding(30522, 100)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=100, bias=True)
        (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (flatt): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
  (fc): Linear(in_features=5000, out_features=30522, bias=True)
)


In [10]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 25 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, device):
    global best_weights, best_correct
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    
    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct/y.size(0)):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    if best_correct < correct:
        best_correct = correct
        best_weights = model.state_dict()


In [11]:
def select_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, train_dataloader, loss_fn, optimizer, device, epochs, learning_rate):
    global best_correct, best_weights
    next_train = True
    
    while next_train:
        for t in range(epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            train_loop(train_dataloader, model, loss_fn, optimizer, device)
            #best_correct, best_weights = test_loop(test_dataloader, model, loss_fn, device)

        print("Done! Saving model...")
        torch.save(model.state_dict(), 'encoder.pth')
        print(f"Best Accuracy: {(100 * best_correct):>0.1f}% Using Learning Rate: {learning_rate}")

        # Preguntar si continuar entrenando
        stop_train = input("Should we continue training? No(Any) Yes(Enter) -> ")
        if stop_train:
            next_train = False
        else:
            # Opción para cambiar el learning rate
            change_lr = input("Change the learning rate? Yes(Any) No(Enter) -> ")
            if change_lr:
                try:
                    learning_rate = float(input("New learning rate: "))
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = learning_rate
                except ValueError:
                    print("Invalid input. Keeping the old learning rate.")

            # Opción para cambiar el número de epochs
            change_epochs = input(f"Change the number of epochs ({epochs})? Yes(Any) No(Enter) -> ")
            if change_epochs:
                try:
                    epochs = int(input("New number of epochs: "))
                except ValueError:
                    print("Invalid input. Keeping the old number of epochs.")

def main(learning_rate, batch_size, epochs):
    global best_correct, best_weights
    
    loss_fn = nn.CrossEntropyLoss()
    device = select_device()

    # Cargar modelo BERT
    model = NeuralNetwork(tokenizer.vocab_size)
    model.to(device)

    # Preguntar si entrenar
    train_on = input("Do you want to train? Yes(y), No(enter): ").strip().lower
    with open("el_quijote.txt", "r", encoding="utf-8") as file:
        contenido = file.read()
    #data = tokenizar(contenido)
    #training_data, test_data = create_train_test_datasets(data)
    #train_dataloader, test_dataloader = create_dataloader(training_data, test_data, batch_size)
    train_dataset = text_dataset(contenido, tokenizer)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    if train_on in ["y", "yes"]:
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        train_model(model, train_dataloader, loss_fn, optimizer, device, epochs, learning_rate)
    else:
        exit()
    #    best_correct, best_weights = test_loop(test_dataloader, model, loss_fn, device)

if __name__ == '__main__':
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    best_correct = 0
    best_weights = None
    learning_rate = 0.0008
    batch_size = 10
    epochs = 10
    #main(learning_rate, batch_size, epochs)

In [12]:
epochs = 2
learning_rate = 0.0001
batch_size = 10

tokenixer = BertTokenizer.from_pretrained('bert-base-uncased')

with open("sample-quij.txt", "r", encoding="utf-8") as file:
        contenido = file.read()
train_dataset = text_dataset(contenido, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
model = NeuralNetwork(tokenizer.vocab_size).to("cuda")

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer, device="cuda")

Token indices sequence length is longer than the specified maximum sequence length for this model (350115 > 512). Running this sequence through the model will result in indexing errors


loss: 10.475163  [   10/315099]
loss: 7.797037  [  260/315099]
loss: 8.731383  [  510/315099]
loss: 7.691641  [  760/315099]
loss: 7.376943  [ 1010/315099]
loss: 8.634453  [ 1260/315099]
loss: 6.517761  [ 1510/315099]
loss: 5.717522  [ 1760/315099]
loss: 6.865970  [ 2010/315099]
loss: 6.063924  [ 2260/315099]
loss: 7.225429  [ 2510/315099]
loss: 6.756115  [ 2760/315099]
loss: 7.339614  [ 3010/315099]
loss: 6.445864  [ 3260/315099]
loss: 6.926076  [ 3510/315099]
loss: 6.398129  [ 3760/315099]
loss: 7.035302  [ 4010/315099]
loss: 7.657014  [ 4260/315099]
loss: 6.077056  [ 4510/315099]
loss: 8.208899  [ 4760/315099]
loss: 5.791764  [ 5010/315099]
loss: 4.782477  [ 5260/315099]
loss: 6.575049  [ 5510/315099]
loss: 5.769012  [ 5760/315099]
loss: 6.557657  [ 6010/315099]
loss: 6.823188  [ 6260/315099]
loss: 6.579680  [ 6510/315099]
loss: 6.421004  [ 6760/315099]
loss: 6.762350  [ 7010/315099]
loss: 5.327163  [ 7260/315099]
loss: 5.831066  [ 7510/315099]
loss: 5.783262  [ 7760/315099]
loss: 7

In [19]:
torch.save(model, 'encoder.pth')
def use_model(model, frase_input,tokenizer, context_size=50):
    frase = tokenizer.encode(frase_input)
    frase = frase[1:-1]
    
    for i in range(context_size-len(frase)):
        frase_zeros = frase.copy()
        for i in range(context_size-len(frase)):
            frase_zeros.append(0)
        frase_zeros = torch.tensor(frase_zeros).reshape(1,50).to("cuda")
        logits = model(frase_zeros)
        y = torch.argmax(logits, dim=1)
        frase += y
    print(tokenizer.decode(frase))
    return frase

In [20]:
frase = ""
use_model(model, frase, tokenizer)

,,, que que, que que que, que que que que, que que que, que, que que que, que,, que que que que, que, que que que, que, que,, que, que,,,


[tensor(1010, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(1010, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device='cuda:0'),
 tensor(10861, device

In [ ]:
tokenizer.decode([2721])